In [118]:
%cd /Users/maayanlab/Documents/projects/human-melanoma-signature
import sys
sys.path.append('pipeline/scripts/')
import pandas as pd
import h5py
import numpy as np
import json, operator, math, requests
from geode_jupies import *
import sklearn.metrics.pairwise as smp
from scipy.spatial.distance import cosine
import seaborn as sns; sns.set(color_codes=True)
import RNAseq as RNA
from collections import Counter

/Users/maayanlab/Documents/projects/human-melanoma-signature


In [35]:
%pwd

'/Users/maayanlab/Documents/projects/human-melanoma-signature'

In [3]:
from Melanoma import *

# Code Development for Melanoma Pipeline
## Define input and output files

In [59]:
# infiles
infile = 's2-signatures.dir/melanoma-geneset-top.json'
outfile = 's2-signatures.dir/melanoma-Enrichr-links-CD600.xls'

In [4]:
# #infiles
# sig = 's2-signatures.dir/melanoma-signatures.txt'

In [5]:
# #infiles
# cpm = 's1-expression_data.dir/melanoma-cpm.txt'
# matches = 's2-expression_data.dir/melanoma-idmatches.txt'

In [7]:
# # Infile & outfile
# infiles = ['s1-expression_data.dir/melanoma-cpm.txt']
# outfile = 's1-expression_data.dir/melanoma-IDstatus.txt'

In [8]:
# # Infile & outfile
# infiles = ['rawdata/human_matrix.h5', 'rawdata/m2.csv']
# outfile = 's1-expression_data.dir/melanoma-cpm.txt'

### Test Code

In [237]:
with open(infile) as openfile:
    dict_df_test = pd.DataFrame(json.load(openfile))

In [238]:
dict_df_test

,1,11,15_a,15_b,15_c,17_a,17_b,17_c,18_a,18_b,...,19_b,19_c,19_d,2_a,2_b,2_c,3,5_a,5_b,5_c
bottom,"[FAT1, ZNF146, JARID2, SLC25A3, STMN1, CDK9, A...","[RABEP2, PABPC4, NF2, KCTD15, CAV2, ARL4C, PLA...","[PDLIM4, AFAP1, TAPBP, LRG1, GNS, SULT1A3, CPD...","[SH3TC1, PLA1A, RPS2P5, OS9, CNTNAP1, SREBF2, ...","[SEC23A, RSU1, CPT1A, DKK3, SKP1, ANXA5, IL13R...","[KIAA0930, LIMD2, LRP3, ABL1, RAB27A, HLA-G, N...","[SORBS1, ADAM23, PKNOX2, MTURN, EPS15, B2M, S1...","[SQLE, CERCAM, MRPL44, MPDZ, FGFR1, ABL2, AIF1...","[TRIO, CYR61, HSPA1A, COL4A1, HLA-A, SRM, HNRN...","[LRP1, EMILIN1, TUFM, SNX1, PTPRJ, PLAT, HIVEP...",...,"[OCIAD2, LIMS1, JTB, CCNB1, PRMT2, CMTM7, HNRN...","[SOD1, NAMPT, TMEM258, LY6K, ANP32A, CDCA4, SR...","[HNRNPH3, DYNC1I2, QKI, TMED3, RDX, ACBD3, TOM...","[AP1S2, BEST1, LONP2, CHST11, ETS1, TPP1, ZNF1...","[LRPAP1, SAMD9, SEMA6D, CFAP61, FBXO10, CALM3,...","[FAM120B, JAZF1, BUB1, ASPM, SCPEP1, SCARF2, T...","[SRSF6, TMEM230, DDX17, TFAP2C, TAX1BP1, MRPL3...","[PPP1CC, KIF20A, AKAP2, CCNI, YWHAH, HNRNPM, A...","[SEPT7, MBP, SUCO, HEXA, UBE2Z, EPAS1, SREBF2,...","[TMOD2, AKAP12, PODXL, RHEB, IDI1, MT-CO2, MLA..."
top,"[COL1A1, COL5A1, COL6A1, FLNA, MMP2, ACTG1, MT...","[PLCG2, HEXIM1, PLLP, PKM, AC010970_2, UBC, PP...","[EFEMP1, MMRN1, VWF, TXNDC5, ALDH1A1, VIM, MT-...","[EFEMP1, MMRN1, TXNDC5, ALDH1A1, IL6ST, MT-CO2...","[EEF1A1, SERPINE1, FTH1, SOD2, MYH9, EEF2, TXN...","[FN1, ENO1, EEF1A1, GAPDH, PKM, TNC, LDHA, TUB...","[GAPDH, VIM, TGFBI, EEF1A1, ANXA2, FLNA, EEF2,...","[SERPINE2, GAPDH, SERPINA3, ENO1, SPARC, PKM, ...","[GDF6, HSPA5, HSP90B1, FTH1, MT-CO1, CALR, HYO...","[HSP90B1, CALR, HSPA5, PDIA3, CPA4, HYOU1, CAL...",...,"[EEF1A1, FTH1, ACTG1, RPLP1, CYR61, RPS28, MT-...","[COL11A1, COL1A1, B2M, MDM2, FTH1, HSPA5, RPLP...","[FTH1, EEF1A1, CYR61, MMACHC, RPS28, H3F3B, MT...","[MT-CO1, EEF1A1, MT-ND2, MT-ND5, MT-ND1, MT-CO...","[A2M, MFSD12, EEF1A1, MT-CO1, FN1, FTH1, AC010...","[TIMP3, TTYH3, MT-CO1, IFI6, PSAP, EEF1A1, A2M...","[SPARC, MT-CO1, FN1, CEMIP, TIMP3, EEF1A2, SLC...","[MT-CO1, MT-CO2, EEF1A1, SOD2, MT-ATP6, MT-ND2...","[FN1, SERPINE2, TIMP3, AC010970_2, VIM, MMP1, ...","[IGFBP5, EEF1A1, FN1, CTNNB1, PHLDA1, GDF15, T..."


In [239]:
dict_top_genes

{'1': {'top': ['COL1A1',
   'COL5A1',
   'COL6A1',
   'FLNA',
   'MMP2',
   'ACTG1',
   'MT-ND4',
   'LGALS1',
   'COL6A2',
   'LMNA',
   'FTH1',
   'S100A6',
   'HPCAL1',
   'HLA-B',
   'CLU',
   'S100A4',
   'HLA-C',
   'PKM',
   'KRT81',
   'FAM129B',
   'ANPEP',
   'HLA-A',
   'MT-CO1',
   'CCND1',
   'SPARC',
   'AHNAK',
   'SLC16A3',
   'ACTB',
   'AGRN',
   'RPS18',
   'STC2',
   'MT-ND2',
   'C3',
   'ATP2A2',
   'COL7A1',
   'VEGFA',
   'SERPINE2',
   'PLAU',
   'TIMP2',
   'PTPN14',
   'CALM2',
   'SLC7A5',
   'TAGLN2',
   'RPL41',
   'YWHAZ',
   'MT-CYB',
   'MYL6',
   'SQSTM1',
   'TNFRSF21',
   'LGALS3BP',
   'EMILIN1',
   'SLC16A1',
   'COL1A2',
   'POSTN',
   'CSNK1D',
   'PRSS23',
   'ANXA2',
   'ACTN4',
   'MMP14',
   'COL15A1',
   'H1F0',
   'LAMA5',
   'ENO1',
   'ALDOA',
   'UBC',
   'CAPN2',
   'MT-ND1',
   'GAPDH',
   'MYL12A',
   'LRP10',
   'VCL',
   'AKR1B1',
   'LIMK1',
   'PTPRN',
   'SPOCK1',
   'PRKCA',
   'NPTX1',
   'SSH1',
   'PCOLCE',
   'CD74',
   'RPL

In [236]:
dict_df = pd.DataFrame(dict_top_genes)
dict_df

,1,11,15_a,15_b,15_c,17_a,17_b,17_c,18_a,18_b,...,19_b,19_c,19_d,2_a,2_b,2_c,3,5_a,5_b,5_c
bottom,"[ZNF146, FAT1, JARID2, SLC25A3, CDK9, STMN1, A...","[ARL4C, PLA2G16, CAV2, C7ORF50, COPS7A, CEMIP,...","[EIF3H, LRG1, AFAP1, SULT1A3, TAPBP, APOL1, GN...","[PLA1A, SH3TC1, RPS2P5, OS9, CNTNAP1, SREBF2, ...","[SEC23A, RSU1, CPT1A, DKK3, SKP1, ANXA5, NUCB1...","[LRP3, LIMD2, HLA-G, ABL1, NRBP2, TGFBR2, GTF2...","[PRKAR1A, ADAM23, PKNOX2, MTURN, S100A1, CORO2...","[MRPL44, FGFR1, ADAMTS2, ABL2, AIF1L, ILVBL, P...","[EIF3CL, TRIO, CYR61, HSPA1A, DHX9, COL4A1, HL...","[USP22, SNX1, PTPRJ, PLAT, LRP1, TUFM, HIVEP3,...",...,"[OCIAD2, LIMS1, JTB, CCNB1, PRMT2, CMTM7, HNRN...","[NAMPT, SOD1, TMEM258, LY6K, ANP32A, CDCA4, SR...","[DYNC1I2, HNRNPH3, QKI, TMED3, RDX, ACBD3, TOM...","[AP1S2, LONP2, BEST1, ETS1, CHST11, TPP1, ZEB2...","[PDIA3, BACE2, PLCG1, FBXO10, LRPAP1, SP110, F...","[FAM120B, DDX3Y, BUB1, SCUBE3, ASPM, S100A1, I...","[TFAP2C, MRPL3, TAX1BP1, HMGB1, RPL24, G3BP2, ...","[PPP1CC, SPHK1, ATP5A1, KIF20A, PPP1R18, CCNI,...","[CYCS, HEXA, SUCO, MBP, SREBF2, EPAS1, UBE2Z, ...","[TMOD2, AKAP12, PODXL, RHEB, IDI1, MLANA, POLR..."
top,"[COL1A1, COL5A1, COL6A1, FLNA, MMP2, ACTG1, MT...","[PLCG2, HEXIM1, PLLP, PKM, AC010970_2, UBC, PP...","[EFEMP1, MMRN1, VWF, TXNDC5, ALDH1A1, VIM, MT-...","[EFEMP1, MMRN1, TXNDC5, ALDH1A1, IL6ST, MT-CO2...","[EEF1A1, SERPINE1, FTH1, SOD2, MYH9, EEF2, TXN...","[FN1, ENO1, EEF1A1, GAPDH, PKM, TNC, LDHA, TUB...","[GAPDH, TGFBI, VIM, ANXA2, EEF1A1, FLNA, EEF2,...","[SERPINE2, GAPDH, SERPINA3, ENO1, SPARC, PKM, ...","[GDF6, HSPA5, HSP90B1, FTH1, MT-CO1, CALR, HYO...","[HSP90B1, CALR, HSPA5, PDIA3, CPA4, HYOU1, CAL...",...,"[EEF1A1, FTH1, ACTG1, RPLP1, CYR61, RPS28, MT-...","[COL11A1, COL1A1, B2M, MDM2, FTH1, HSPA5, RPLP...","[FTH1, EEF1A1, CYR61, MMACHC, RPS28, H3F3B, MT...","[MT-CO1, EEF1A1, MT-ND2, MT-ND5, MT-ND1, MT-CO...","[A2M, MFSD12, EEF1A1, MT-CO1, FN1, FTH1, AC010...","[TIMP3, TTYH3, MT-CO1, IFI6, PSAP, EEF1A1, A2M...","[SPARC, MT-CO1, FN1, CEMIP, TIMP3, EEF1A2, SLC...","[MT-CO1, MT-CO2, EEF1A1, SOD2, MT-ATP6, MT-ND2...","[FN1, SERPINE2, TIMP3, VIM, AC010970_2, MMP1, ...","[IGFBP5, EEF1A1, FN1, CTNNB1, PHLDA1, GDF15, T..."


In [165]:
cn=dict_df.columns.get_values().tolist()

In [175]:
def enrichr_result_to_df(genes, meta, gmt):
    res = enrichr_result(genes, meta=meta, gmt=gmt)
    columns = [
    'Rank', 
    'Term name', 
    'P-value', 
    'Z-score', 
    'Combined score', 
    'Overlapping genes', 
    'Adjusted p-value', 
    'Old p-value', 
    'Old adjusted p-value'
    ]    
    df = pd.DataFrame.from_records(res[gmt], columns=columns)
    return df

In [226]:
import time

In [228]:
def submit_enrichr_geneset(geneset):
	ENRICHR_URL = 'http://amp.pharm.mssm.edu/Enrichr/addList'
	genes_str = '\n'.join(geneset)
	payload = {
	'list': (None, genes_str),
	}
	response = requests.post(ENRICHR_URL, files=payload)
	if not response.ok:
		raise Exception('Error analyzing gene list')
	data = json.loads(response.text)
	return data

def get_enrichr_results(user_list_id, gene_set_libraries=['GO_Biological_Process_2018'], overlappingGenes=True, geneset=None):
	ENRICHR_URL = 'http://amp.pharm.mssm.edu/Enrichr/enrich'
	query_string = '?userListId=%s&backgroundType=%s'
	results = []
	for gene_set_library in gene_set_libraries:
		response = requests.get(
                    ENRICHR_URL +
                   	query_string % (user_list_id, gene_set_library)
                )
		if not response.ok:
			raise Exception('Error fetching enrichment results')

		data = json.loads(response.text)
		resultDataframe = pd.DataFrame(data[gene_set_library], columns=[
		                               'rank', 'term_name', 'pvalue', 'zscore', 'combined_score', 'overlapping_genes', 'FDR', 'old_pvalue', 'old_FDR'])
		selectedColumns = ['term_name', 'zscore', 'combined_score', 'pvalue', 'FDR'] if not overlappingGenes else [
			'term_name', 'zscore', 'combined_score', 'FDR', 'pvalue', 'overlapping_genes']
		resultDataframe = resultDataframe.loc[:, selectedColumns]
		resultDataframe['gene_set_library'] = gene_set_library
		results.append(resultDataframe)
	concatenatedDataframe = pd.concat(results)
	if geneset:
		concatenatedDataframe['geneset'] = geneset
	return concatenatedDataframe

def run_enrichr(genes, genesets=['GO_Biological_Process_2018']):
	ids = submit_enrichr_geneset(genes)
	time.sleep(0.5)
	results = get_enrichr_results(ids['userListId'], genesets)
	return results

In [229]:
# Create a 4th dictionary 
enr_res = defaultdict(dict)
# Iterate through dictionary, create gene lists and save as 'genes'
for signature in dict_top_genes:
    print('Doing '+signature)
    for direction in ('top', 'bottom'):
        genes = dict_top_genes[signature][direction] # value = dict [1st key] [2nd key]
        enr_res[signature][direction] = run_enrichr(genes)

In [231]:
# freeze default dict for readonly1`
enr_res.default_factory = None

In [232]:
enr_res

defaultdict(None,
            {'1': {'top':                                               term_name    zscore  \
              0                  protein targeting to ER (GO:0045047) -2.085357   
              1        extracellular matrix organization (GO:0030198) -1.646511   
              2     SRP-dependent cotranslational protein targetin... -1.839292   
              3     nuclear-transcribed mRNA catabolic process, no... -1.997175   
              4                 neutrophil degranulation (GO:0043312) -2.004472   
              5             neutrophil mediated immunity (GO:0002446) -1.930822   
              6                      viral transcription (GO:0019083) -1.784887   
              7     cotranslational protein targeting to membrane ... -1.369027   
              8                            viral process (GO:0016032) -1.686963   
              9                    viral gene expression (GO:0019080) -1.579705   
              10                  rRNA metabolic process 

In [234]:
c = Counter()
for i in [enr_res[signature]['top'] for signature in enr_res.keys()]:
    c.update(i['term_name'].iloc[:5])

In [235]:
c.most_common()

[('protein targeting to ER (GO:0045047)', 17),
 ('SRP-dependent cotranslational protein targeting to membrane (GO:0006614)',
  17),
 ('nuclear-transcribed mRNA catabolic process, nonsense-mediated decay (GO:0000184)',
  17),
 ('viral transcription (GO:0019083)', 13),
 ('peptide biosynthetic process (GO:0043043)', 8),
 ('viral process (GO:0016032)', 5),
 ('neutrophil degranulation (GO:0043312)', 4),
 ('extracellular matrix organization (GO:0030198)', 3),
 ('neutrophil mediated immunity (GO:0002446)', 3),
 ('regulated exocytosis (GO:0045055)', 2),
 ('platelet degranulation (GO:0002576)', 2),
 ('rRNA metabolic process (GO:0016072)', 2),
 ('mitochondrial ATP synthesis coupled proton transport (GO:0042776)', 1),
 ('ribosomal small subunit export from nucleus (GO:0000056)', 1),
 ('positive regulation of nitric oxide biosynthetic process (GO:0045429)', 1),
 ('positive regulation of nitric oxide metabolic process (GO:1904407)', 1),
 ('ER to Golgi vesicle-mediated transport (GO:0006888)', 1),
 

In [ ]:
[enr_res[signature]['top'] for signature in enr_res.keys()]

In [176]:
# Create a 3rd dictionary 
from collections import defaultdict
results = defaultdict(dict)
# Iterate through dictionary, create gene lists and save as 'genes'
for signature in dict_top_genes:
    for direction in ('top', 'bottom'):
        genes = dict_top_genes[signature][direction] # value = dict [1st key] [2nd key]
        results[signature][direction] = enrichr_result_to_df(genes,'','GO_Biological_Process_2018')
        #for signature in list(dict_top_genes.keys())[0]:

In [178]:
# freeze default dict for readonly1
results.default_factory = None

In [ ]:
c = Counter()
for i in [results[signature]['top'] for signature in results.keys()]
for i in [results['1']['top']]: 
    c.update(i['Term name'].iloc[:5])
[results[signature]['top'] for signature in results.keys()]

In [209]:
# if creating a list instead of dictionary
test = []
for signature in dict_top_genes:
    for direction in ('top', 'bottom'):
        genes = dict_top_genes[signature][direction] # value = dict [1st key] [2nd key]
        genes_df = pd.DataFrame(enrichr_result_to_df(genes,'','GO_Biological_Process_2018'))
        #specify names of each geneset (which study and top/bottom they are from)
        test.append([signature, direction, genes_df])

In [201]:
# Number of studies to be processed
n = len(study_names)
i = 0
for study in study_names[:]:
    
df_1_top=pd.DataFrame(results['1']['top'])

In [ ]:
for study, d1 in D.items():
    for top, v in d1.items():
        

In [169]:
wanted_keys = dict_df.columns.get_values().tolist() # The keys you want
dict((k, results[k]) for k in wanted_keys if k in results)

TypeError: list indices must be integers or slices, not str

In [198]:
c = Counter()
for i in [results['1']['top']]: 
    c.update(i['Term name'].iloc[:5])

KeyError: '11'

In [197]:
c.most_common()

[('protein targeting to ER (GO:0045047)', 1),
 ('extracellular matrix organization (GO:0030198)', 1),
 ('SRP-dependent cotranslational protein targeting to membrane (GO:0006614)',
  1),
 ('nuclear-transcribed mRNA catabolic process, nonsense-mediated decay (GO:0000184)',
  1),
 ('neutrophil degranulation (GO:0043312)', 1)]

In [57]:
for top, d in dict_top_genes.items():    
    genes = dict_top_genes[top].top()
    link = RNAseq.enrichr_link(genes, top)
    print (top)
    display_link(link)

AttributeError: 'dict' object has no attribute 'top'

In [ ]:
c = Counter()
for df_ in [df, df2]: 
    c.update(df_['Term name'].iloc[:5])

In [65]:
## Generate Enrichr links for up/down genes in an Excel file with the gene sets.
#enrichr_result_file = os.path.join(os.environ['WORKDIR'], 'Enrichr_links_CD600.xls')
RNA.dict2xls_with_vals(dict_top_genes, ['study'], outfile)

TypeError: 'dict_values' object does not support indexing